d-sandbox

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px; height: 163px">
</div>

-sandbox
<img src="https://files.training.databricks.com/images/Apache-Spark-Logo_TM_200px.png" style="float: left: margin: 20px"/>

# Structured Streaming Concepts Lab

## Instructions
* Insert solutions wherever it says `FILL_IN`
* Feel free to copy/paste code from the previous notebook, where applicable
* Run test cells to verify that your solution is correct

## Prerequisites
* Web browser: current versions of Google Chrome, Firefox, Safari, Microsoft Edge and 
Internet Explorer 11 on Windows 7, 8, or 10 (see <a href="https://docs.databricks.com/user-guide/supported-browsers.html#supported-browsers#" target="_blank">Supported Web Browsers</a>)
* Databricks Runtime 4.2 or greater
* Completed courses Spark-SQL, DataFrames or ETL-Part 1 from <a href="https://academy.databricks.com/" target="_blank">Databricks Academy</a>, or have similar knowledge


## CAUTION
* Streaming jobs will quickly exhaust your disk quota, if you are using Community Edition. If you find yourself running out of disk space, 
  you can [mount your own S3 bucket](https://docs.databricks.com/spark/latest/data-sources/aws/amazon-s3.html#mount-an-s3-bucket) and change
  your streaming paths accordingly.

<h2><img src="https://files.training.databricks.com/images/105/logo_spark_tiny.png"> Getting Started</h2>

Run the following cell to configure our "classroom."

In [4]:
%run "../Includes/Classroom-Setup"

<h2><img src="https://files.training.databricks.com/images/105/logo_spark_tiny.png"> Exercise 1: Read Stream</h2>

The dataset used in this exercise consists of flight information about flights from/to various airports in 2007.

Run the following cell to see what the streaming data will look like.

In [6]:
display(
  spark.read.parquet("dbfs:/mnt/training/asa/flights/2007-01-stream.parquet/part-00000-tid-9167815511861375854-22d81a30-d5b4-43d0-9216-0c20d14c3f54-178-c000.snappy.parquet")
)

DepartureAt,FlightDate,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
2007-01-01T00:01:00.000+0000,2007-01-01,00:01,22:05,07:24,05:33,NW,96,N593NW,323,328,293,111,116,HNL,SEA,2677,5,25,0,null,0,111,0,0,0,0
2007-01-01T00:01:00.000+0000,2007-01-01,00:01,23:45,07:22,06:50,DL,704,N607DL,261,245,228,32,16,LAX,CVG,1900,7,26,0,null,0,16,0,16,0,0
2007-01-01T00:01:00.000+0000,2007-01-01,00:01,23:30,07:05,06:42,DL,748,N380DA,244,252,221,23,31,SEA,CVG,1964,7,16,0,null,0,23,0,0,0,0
2007-01-01T00:01:00.000+0000,2007-01-01,00:01,22:45,08:44,07:10,AA,192,N632AA,343,325,320,94,76,LAX,BOS,2611,6,17,0,null,0,51,0,18,0,25
2007-01-01T00:01:00.000+0000,2007-01-01,00:01,23:55,06:13,06:15,CO,434,N76504,252,260,232,-2,6,PHX,EWR,2133,5,15,0,null,0,0,0,0,0,0
2007-01-01T00:03:00.000+0000,2007-01-01,00:03,23:55,02:48,02:32,YV,2876,N921FJ,105,97,80,16,8,LAS,ELP,584,4,21,0,null,0,16,0,0,0,0
2007-01-01T00:04:00.000+0000,2007-01-01,00:04,00:10,07:34,07:38,NW,336,N523US,270,268,233,-4,-6,LAX,DTW,1979,13,24,0,null,0,0,0,0,0,0
2007-01-01T00:04:00.000+0000,2007-01-01,00:04,23:57,07:32,07:16,US,49,N828AW,268,259,243,16,7,LAS,DCA,2089,3,22,0,null,0,7,0,9,0,0
2007-01-01T00:04:00.000+0000,2007-01-01,00:04,20:50,02:04,23:00,AA,2376,N509AA,120,130,99,184,194,DFW,ORD,802,12,9,0,null,0,0,0,0,0,184
2007-01-01T00:05:00.000+0000,2007-01-01,00:05,23:55,05:30,05:27,CO,229,N17344,205,212,180,3,10,DEN,EWR,1605,9,16,0,null,0,0,0,0,0,0


Start by reading a stream. 

For this step you will need to:
0. Starting with `spark`, an instance of `SparkSession`, and get the `DataStreamReader`
0. Make sure to only consume only 1 file per trigger
0. Specify the stream's schema using the instance `dataSchema` (already provided for you)
0. Use `dsr.json()` to specify the stream's file type and source directory, `dataPath` 

When you are done, run the TEST cell that follows to verify your results.

In [8]:
# TODO
dataSchema = "DepartureAt timestamp, FlightDate string, DepTime string, CRSDepTime string, ArrTime string, CRSArrTime string, UniqueCarrier string, FlightNum integer, TailNum string, ActualElapsedTime string, CRSElapsedTime string, AirTime string, ArrDelay string, DepDelay string, Origin string, Dest string, Distance string, TaxiIn string, TaxiOut string, Cancelled integer, CancellationCode string, Diverted integer, CarrierDelay string, WeatherDelay string, NASDelay string, SecurityDelay string, LateAircraftDelay string"

dataPath = "dbfs:/mnt/training/asa/flights/2007-01-stream.parquet"

initialDF = (spark
  .readStream   # Get a DataStreamReader
  .option("maxFilesPerTrigger",1)   # Force processing of only 1 file per trigger 
  .schema(dataSchema)# Use the schema "dataSchema"
  .parquet(dataPath)   # Read in stream's file type and source directory
)
display(initialDF)

DepartureAt,FlightDate,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
2007-01-01T00:01:00.000+0000,2007-01-01,00:01,22:05,07:24,05:33,NW,96,N593NW,323,328,293,111,116,HNL,SEA,2677,5,25,0,null,0,111,0,0,0,0
2007-01-01T00:01:00.000+0000,2007-01-01,00:01,23:45,07:22,06:50,DL,704,N607DL,261,245,228,32,16,LAX,CVG,1900,7,26,0,null,0,16,0,16,0,0
2007-01-01T00:01:00.000+0000,2007-01-01,00:01,23:30,07:05,06:42,DL,748,N380DA,244,252,221,23,31,SEA,CVG,1964,7,16,0,null,0,23,0,0,0,0
2007-01-01T00:01:00.000+0000,2007-01-01,00:01,22:45,08:44,07:10,AA,192,N632AA,343,325,320,94,76,LAX,BOS,2611,6,17,0,null,0,51,0,18,0,25
2007-01-01T00:01:00.000+0000,2007-01-01,00:01,23:55,06:13,06:15,CO,434,N76504,252,260,232,-2,6,PHX,EWR,2133,5,15,0,null,0,0,0,0,0,0
2007-01-01T00:03:00.000+0000,2007-01-01,00:03,23:55,02:48,02:32,YV,2876,N921FJ,105,97,80,16,8,LAS,ELP,584,4,21,0,null,0,16,0,0,0,0
2007-01-01T00:04:00.000+0000,2007-01-01,00:04,00:10,07:34,07:38,NW,336,N523US,270,268,233,-4,-6,LAX,DTW,1979,13,24,0,null,0,0,0,0,0,0
2007-01-01T00:04:00.000+0000,2007-01-01,00:04,23:57,07:32,07:16,US,49,N828AW,268,259,243,16,7,LAS,DCA,2089,3,22,0,null,0,7,0,9,0,0
2007-01-01T00:04:00.000+0000,2007-01-01,00:04,20:50,02:04,23:00,AA,2376,N509AA,120,130,99,184,194,DFW,ORD,802,12,9,0,null,0,0,0,0,0,184
2007-01-01T00:05:00.000+0000,2007-01-01,00:05,23:55,05:30,05:27,CO,229,N17344,205,212,180,3,10,DEN,EWR,1605,9,16,0,null,0,0,0,0,0,0


In [9]:
# TEST - Run this cell to test your solution.
schemaStr = str(initialDF.schema)

dbTest("SS-02-schema-01", True, "(DepartureAt,TimestampType,true)" in schemaStr) 
dbTest("SS-02-schema-02", True, "(FlightDate,StringType,true)" in schemaStr) 
dbTest("SS-02-schema-03", True, "(DepTime,StringType,true)" in schemaStr) 
dbTest("SS-02-schema-04", True, "(CRSDepTime,StringType,true)" in schemaStr) 
dbTest("SS-02-schema-05", True, "(ArrTime,StringType,true)" in schemaStr) 
dbTest("SS-02-schema-06", True, "(CRSArrTime,StringType,true)" in schemaStr) 
dbTest("SS-02-schema-07", True, "(UniqueCarrier,StringType,true)" in schemaStr) 
dbTest("SS-02-schema-08", True, "(FlightNum,IntegerType,true)" in schemaStr) 
dbTest("SS-02-schema-09", True, "(TailNum,StringType,true)" in schemaStr) 
dbTest("SS-02-schema-10", True, "(ActualElapsedTime,StringType,true)" in schemaStr) 
dbTest("SS-02-schema-11", True, "(CRSElapsedTime,StringType,true)" in schemaStr) 
dbTest("SS-02-schema-12", True, "(AirTime,StringType,true)" in schemaStr) 
dbTest("SS-02-schema-13", True, "(ArrDelay,StringType,true)" in schemaStr) 
dbTest("SS-02-schema-14", True, "(DepDelay,StringType,true)" in schemaStr) 
dbTest("SS-02-schema-15", True, "(Origin,StringType,true)" in schemaStr) 
dbTest("SS-02-schema-16", True, "(Dest,StringType,true)" in schemaStr) 
dbTest("SS-02-schema-17", True, "(Distance,StringType,true)" in schemaStr) 
dbTest("SS-02-schema-18", True, "(TaxiIn,StringType,true)" in schemaStr) 
dbTest("SS-02-schema-19", True, "(TaxiOut,StringType,true)" in schemaStr) 
dbTest("SS-02-schema-20", True, "(Cancelled,IntegerType,true)" in schemaStr) 
dbTest("SS-02-schema-21", True, "(CancellationCode,StringType,true)" in schemaStr) 
dbTest("SS-02-schema-22", True, "(Diverted,IntegerType,true)" in schemaStr) 
dbTest("SS-02-schema-23", True, "(CarrierDelay,StringType,true)" in schemaStr) 
dbTest("SS-02-schema-24", True, "(WeatherDelay,StringType,true)" in schemaStr) 
dbTest("SS-02-schema-25", True, "(NASDelay,StringType,true)" in schemaStr) 
dbTest("SS-02-schema-26", True, "(SecurityDelay,StringType,true)" in schemaStr) 
dbTest("SS-02-schema-27", True, "(LateAircraftDelay,StringType,true)" in schemaStr) 

print("Tests passed!")

Tests passed!

-sandbox

<h2><img src="https://files.training.databricks.com/images/105/logo_spark_tiny.png"> Exercise 2: Calculate the total of all delays</h2>

We want to calculate (and later graph) the total delay of each flight
0. Start with `initialDF` from the previous exercise 
0. Convert the following columns from `String` to `Integer`: `CarrierDelay`, `WeatherDelay`, `NASDelay`, `SecurityDelay` and `LateAircraftDelay`
0. Add the column `TotalDelay` which is the sum of the other 5 delays
0. Filter the flights by `UniqueCarrier` down to the carriers **AS**, **AQ**, **HA** and **F9**
0. Filter the results to non-zero delay's (`TotalDelay` > 0)
0. Assign the final DataFrame to `delaysDF`

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> The `display()` function will only plot the first 1000 records. By limiting ourselves to four carriers and non-zero delays, we can help to ensure that we get a reasonable demonstration of a live plot.

In [11]:

from pyspark.sql.functions import col
delaysDF = (initialDF
  .withColumn("CarrierDelay",col("CarrierDelay").cast("Integer"))  # Convert CarrierDelay to an Integer
  .withColumn("WeatherDelay",col("WeatherDelay").cast("Integer"))  # Convert WeatherDelay to an Integer
  .withColumn("NASDelay",col("NASDelay").cast("Integer"))  # Convert NASDelay to an Integer
  .withColumn("SecurityDelay",col("SecurityDelay").cast("Integer"))   # Convert SecurityDelay to an Integer
  .withColumn("LateAircraftDelay",col("LateAircraftDelay").cast("Integer"))  # Convert LateAircraftDelay to an Integer
  .withColumn("TotalDelay",sum(col(x) for x in initialDF.columns[22:]))   # Sum all five as TotalDelay   
  
  .where((col("UniqueCarrier")=="AS") | (col("UniqueCarrier")=="AQ") | (col("UniqueCarrier")=="HA") | (col("UniqueCarrier")=="F9"))  # Filter UniqueCarrier to only "AS", "AQ", "HA" and "F9"
  .filter(col("TotalDelay")!=0)  # TotalDelay to non-zero values
)

display(delaysDF)

DepartureAt,FlightDate,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,TotalDelay
2007-01-01T00:06:00.000+0000,2007-01-01,00:06,23:55,05:51,05:35,F9,380,N948FR,225,220,208,16,11,DEN,FLL,1703,4,13,0,null,0,11,0,5,0,0,16
2007-01-01T00:34:00.000+0000,2007-01-01,00:34,23:55,01:32,00:52,AS,191,N765AS,58,57,37,40,39,ANC,FAI,261,4,17,0,null,0,39,0,1,0,0,40
2007-01-01T07:05:00.000+0000,2007-01-01,07:05,07:00,08:39,08:23,AS,583,N622AS,94,83,56,16,5,LAX,SFO,337,21,17,0,null,0,0,0,16,0,0,16
2007-01-01T08:06:00.000+0000,2007-01-01,08:06,08:10,15:45,15:25,HA,10,N590HA,459,435,432,20,-4,HNL,LAX,2556,10,17,0,null,0,20,0,0,0,0,20
2007-01-01T08:34:00.000+0000,2007-01-01,08:34,07:25,10:39,09:30,AS,351,N302AS,125,125,109,69,69,SFO,SEA,679,3,13,0,null,0,69,0,0,0,0,69
2007-01-01T08:35:00.000+0000,2007-01-01,08:35,08:30,11:48,11:30,AS,595,N972AS,193,180,168,18,5,SAN,SEA,1050,6,19,0,null,0,0,0,18,0,0,18
2007-01-01T09:11:00.000+0000,2007-01-01,09:11,08:05,11:10,10:22,AS,232,N786AS,119,137,98,48,66,SEA,SFO,679,5,16,0,null,0,48,0,0,0,0,48
2007-01-01T09:29:00.000+0000,2007-01-01,09:29,09:00,13:57,13:30,AS,196,N645AS,208,210,187,27,29,ANC,SEA,1449,7,14,0,null,0,27,0,0,0,0,27
2007-01-01T09:42:00.000+0000,2007-01-01,09:42,09:15,12:29,11:56,AS,663,N649AS,167,161,146,33,27,LAS,SEA,866,3,18,0,null,0,27,0,6,0,0,33
2007-01-01T09:47:00.000+0000,2007-01-01,09:47,08:25,12:54,12:00,HA,35,N594HA,187,215,166,54,82,PHX,HNL,2917,3,18,0,null,0,54,0,0,0,0,54


In [12]:
 #TEST - Run this cell to test your solution.
schemaStr = str(delaysDF.schema)

dbTest("SS-02-schema-01", True, "(UniqueCarrier,StringType,true)" in schemaStr) 
dbTest("SS-02-schema-02", True, "(TotalDelay,IntegerType,true)" in schemaStr) 
dbTest("SS-02-schema-03", True, "(CarrierDelay,IntegerType,true)" in schemaStr) 
dbTest("SS-02-schema-04", True, "(WeatherDelay,IntegerType,true)" in schemaStr) 
dbTest("SS-02-schema-05", True, "(NASDelay,IntegerType,true)" in schemaStr) 
dbTest("SS-02-schema-06", True, "(SecurityDelay,IntegerType,true)" in schemaStr) 
dbTest("SS-02-schema-07", True, "(LateAircraftDelay,IntegerType,true)" in schemaStr) 
dbTest("SS-02-schema-08", True, "(DepartureAt,TimestampType,true)" in schemaStr) 

print("Tests passed!")

Tests passed!

-sandbox
<h2><img src="https://files.training.databricks.com/images/105/logo_spark_tiny.png"> Exercise 3: Plot a LIVE graph</h2>

Plot `delaysDF` and give the stream the name "delays_python"

Once the data is loaded, render a line graph with 
* **Keys** is set to `DepartureAt`
* **Series groupings** is set to `UniqueCarrier`
* **Values** is set to `TotalDelay`

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> Because of the `display()` function's 1000 record limit, the stream will appear to stop shortly after January 5th.

In [14]:
myStream="delays_python"
display(delaysDF, streamName=myStream)

DepartureAt,FlightDate,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,TotalDelay
2007-01-01T00:06:00.000+0000,2007-01-01,00:06,23:55,05:51,05:35,F9,380,N948FR,225,220,208,16,11,DEN,FLL,1703,4,13,0,null,0,11,0,5,0,0,16
2007-01-01T00:34:00.000+0000,2007-01-01,00:34,23:55,01:32,00:52,AS,191,N765AS,58,57,37,40,39,ANC,FAI,261,4,17,0,null,0,39,0,1,0,0,40
2007-01-01T07:05:00.000+0000,2007-01-01,07:05,07:00,08:39,08:23,AS,583,N622AS,94,83,56,16,5,LAX,SFO,337,21,17,0,null,0,0,0,16,0,0,16
2007-01-01T08:06:00.000+0000,2007-01-01,08:06,08:10,15:45,15:25,HA,10,N590HA,459,435,432,20,-4,HNL,LAX,2556,10,17,0,null,0,20,0,0,0,0,20
2007-01-01T08:34:00.000+0000,2007-01-01,08:34,07:25,10:39,09:30,AS,351,N302AS,125,125,109,69,69,SFO,SEA,679,3,13,0,null,0,69,0,0,0,0,69
2007-01-01T08:35:00.000+0000,2007-01-01,08:35,08:30,11:48,11:30,AS,595,N972AS,193,180,168,18,5,SAN,SEA,1050,6,19,0,null,0,0,0,18,0,0,18
2007-01-01T09:11:00.000+0000,2007-01-01,09:11,08:05,11:10,10:22,AS,232,N786AS,119,137,98,48,66,SEA,SFO,679,5,16,0,null,0,48,0,0,0,0,48
2007-01-01T09:29:00.000+0000,2007-01-01,09:29,09:00,13:57,13:30,AS,196,N645AS,208,210,187,27,29,ANC,SEA,1449,7,14,0,null,0,27,0,0,0,0,27
2007-01-01T09:42:00.000+0000,2007-01-01,09:42,09:15,12:29,11:56,AS,663,N649AS,167,161,146,33,27,LAS,SEA,866,3,18,0,null,0,27,0,6,0,0,33
2007-01-01T09:47:00.000+0000,2007-01-01,09:47,08:25,12:54,12:00,HA,35,N594HA,187,215,166,54,82,PHX,HNL,2917,3,18,0,null,0,54,0,0,0,0,54


In [15]:
# TEST - Run this cell to test your solution.
count = 0
for s in spark.streams.active:
  if (s.name == "delays_python"):
    count = count + 1

dbTest("SS-02-runningCount", 1, count)

print("Tests passed!")

Tests passed!

When you are done, stop the stream:

In [17]:
for s in spark.streams.active:
  s.stop()

<h2><img src="https://files.training.databricks.com/images/105/logo_spark_tiny.png"> Exercise 4: Write Stream</h2>

Write the stream to an in-memory table
0. Use appropriate `format`
0. For this exercise, we want to append new records to the results table
0. Configure a 15 second trigger
0. Name the query "delays_python"
0. Start the query
0. Assign the query to `delayQuery`

In [19]:
basePath = userhome + "/structured-streaming-concepts/python" 
dbutils.fs.mkdirs(basePath)                                   
outputPathDir = basePath + "/output.parquet" 
delayQuery = (delaysDF 
 .writeStream  # From the DataFrame get the DataStreamWriter
 .format("memory")  # Specify the sink format as "memory"
 .outputMode("append")  # Configure the output mode as "append"
 .queryName("delays_python")  # Name the query "delays_python"
 .trigger(processingTime="15 seconds")  # Use a 15 second trigger
 .start(outputPathDir)  # Start the query
)

In [20]:
# TEST - Run this cell to test your solution.
dbTest("SS-02-isActive", True, delayQuery.isActive)
dbTest("SS-02-name", "delays_python", delayQuery.name)
# The query's trigger is not available via the Python API

print("Tests passed!")

Tests passed!

Wait until stream is done initializing...

In [22]:
untilStreamIsReady("delays_python")

The stream is active and ready.

<h2><img src="https://files.training.databricks.com/images/105/logo_spark_tiny.png"> Exercise 5: Stop streaming jobs</h2>

Before we can conclude, we need to shut down all active streams.

In [24]:
# TODO
for stream in spark.streams.active:              # Iterate over all active streams
  print("stopping {}".format(stream.name))  # A little console output
  stream.stop()                            # Stop the stream

stopping delays_python

In [25]:
# TEST - Run this cell to test your solution.
dbTest("SS-02-numActiveStreams", 0, len(spark.streams.active))

print("Tests passed!")

Tests passed!

<h2><img src="https://files.training.databricks.com/images/105/logo_spark_tiny.png"> Next Steps</h2>

Start the next lesson, [Time Windows]($../SS 03 - Time Windows).

-sandbox
&copy; 2019 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="http://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="http://help.databricks.com/">Support</a>